In [ ]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
model_MobilNetV2  = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
model_MobilNetV2.trainable = False

# ImageNet labels
decode_predictions_MobilNetV2 = tf.keras.applications.mobilenet_v2.decode_predictions

In [ ]:
model_MobilNet  = tf.keras.applications.MobileNet(include_top=True,weights='imagenet')
model_MobilNet.trainable = False

In [ ]:
model_ResNet101 = tf.keras.applications.ResNet101(include_top=True,
                                                     weights='imagenet')
model_ResNet101.trainable = False
decode_predictions_resnet = tf.keras.applications.resnet.decode_predictions
# ImageNet labels

In [ ]:
model_inception_v3 = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet')
model_inception_v3.trainable = False

decode_predictions_inception_v3 = tf.keras.applications.inception_v3.decode_predictions

In [ ]:
model_InceptionResNetV2 = tf.keras.applications.InceptionResNetV2(include_top=True,
                                                     weights='imagenet')
model_InceptionResNetV2.trainable = False



In [ ]:
model_VGG16 = tf.keras.applications.VGG16(include_top=True, weights='imagenet')
model_VGG16.trainable = False

In [ ]:
model_xception = tf.keras.applications.Xception(include_top=True, weights='imagenet')
model_xception.trainable = False

In [ ]:
model_ResNet152 = tf.keras.applications.ResNet152(include_top=True,
                                                     weights='imagenet')
model_ResNet152.trainable = False

decode_predictions_resnet = tf.keras.applications.resnet.decode_predictions

In [ ]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = image/255
    image = tf.image.resize(image, (224, 224))
    image = image[None, ...]
    return image

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
    return decode_predictions_MobilNetV2(probs, top=1)[0][0]

In [ ]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image_raw = tf.io.read_file(image_path)
image = tf.image.decode_image(image_raw)

image = preprocess(image)
image_probs = model_MobilNetV2.predict(image)

In [ ]:
plt.figure()
plt.imshow(image[0])
_, image_class, class_confidence = get_imagenet_label(image_probs)
plt.title('{} : {:.2f}% Confidence'.format(image_class, class_confidence*100))
print(image[0])
plt.show()

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model_MobilNetV2(input_image)
        loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

In [ ]:
labrador_retriever_index = 208
label = tf.one_hot(labrador_retriever_index, image_probs.shape[-1])
label = tf.reshape(label, (1, image_probs.shape[-1]))

perturbations = create_adversarial_pattern(image, label)
plt.imshow(perturbations[0])

In [ ]:
image.shape

In [ ]:
def display_images(image, description):
    _, label, confidence = get_imagenet_label(model_MobilNetV2.predict(image))
    plt.figure()
    plt.imshow(image[0])
    plt.title('{} \n {} : {:.2f}% Confidence'.format(description,
                                                   label, confidence*100))
    plt.show()

In [ ]:
epsilons = [0, 0.01, 0.1, 0.15]
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                for eps in epsilons]

for i, eps in enumerate(epsilons):
    adv_x = image + eps*perturbations
    adv_x = tf.clip_by_value(adv_x, 0, 1)
    display_images(adv_x, descriptions[i])

In [ ]:
def sneaky_adversarial_MI_FGSM( n, y, steps, eta, lam , model):
    print(y.shape)
    x = (y + np.ones((y.shape)))/2
    x = tf.image.resize(x, (299, 299))
    x = x[None, ...]
    print(x.shape)
    g = np.zeros((1, 299,299,3))
    g=tf.convert_to_tensor(g,dtype='float32')
    goal = np.zeros((1000, 1))
    goal[n]=1
    goal=goal.reshape((1,1000))
    goal=tf.convert_to_tensor(goal)
    alpha=eta/steps
    for i in range(steps):
        d = grad(x,goal,model)
        d_1=tf.norm(d, ord=1)
        g = lam * g + d/d_1
        g_2=tf.norm( g , ord = 1)
        x = x - alpha* tf.sign(g)
    return x


In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
def grad(input_image, input_label,model):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_object(input_label, prediction)
        

  # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    #Get the sign of the gradients to create the perturbation
    #signed_grad = tf.sign(gradient)
    return gradient

In [ ]:
import cv2

In [ ]:
import numpy as np
description = ' ' 

adv_ex = sneaky_adversarial_MI_FGSM(345, images[12] , 10 , 0.05 , 0  ,model_MobilNetV2 )
print(np.argmax(model_MobilNetV2.predict(adv_ex)))
#x = (adv_ex[0] + np.ones((adv_ex[0].shape)))/2
plt.imshow(adv_ex[0])


In [ ]:
true_classes[12]-1

In [ ]:
def sneaky_adversarial_FGSM( n, y , eta,  model):
    x = (y + np.ones((y.shape)))/2
    x = tf.image.resize(x, (299, 299))
    x = x[None, ...]
    goal = np.zeros((1000, 1))
    goal[n]=1
    goal=goal.reshape((1,1000))
    goal=tf.convert_to_tensor(goal)
    adv = x - eta * tf.sign(grad(x , goal , model ))
    return adv


In [ ]:
adv_ex = sneaky_adversarial_FGSM(2, images[62] , 0.05  ,model_InceptionResNetV2 )
print(np.argmax(model_InceptionResNetV2.predict(adv_ex)))
#x = (adv_ex[0] + np.ones((adv_ex[0].shape)))/2
plt.imshow(adv_ex[0])


In [ ]:
show_image(images[0])

In [ ]:
def grad_TV(input_image):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        func = TV(input_image)
    gradient = tape.gradient(func, input_image)
    #Get the sign of the gradients to create the perturbation
    #signed_grad 
    return gradient

In [ ]:
x = (images[123] + np.ones((images[123].shape)))/2
x = tf.image.resize(x, (299, 299))
x = x[None, ...]
print(np.argmax(model_inception_v3.predict(x)))
show_image(images[123])

In [ ]:
images.shape

In [ ]:
def sneaky_adversarial( n, x_target, steps, eta, lam , model):

    #x=-1.*np.random.sample((1,224,224,3))
    x=-1.*np.ones((1,224,224,3))
    M = np.zeros((1,224,224,3))
    for i in range(25):
        for j in range(3):
            for k in range(25):
                M[0,i+45,k+45,j]=1. 
    M = tf.convert_to_tensor(M,dtype='float32')
    x=tf.convert_to_tensor(x,dtype='float32')
    x_t = (x_target + np.ones((x_target.shape)))/2
    x_t = tf.image.resize(x_t, (224, 224))
    #x_t = x_t[None, ...]
    #x_t = cm(x_t,M1)
    goal = np.zeros((1000, 1))
    goal[n]=1    
    goal=goal.reshape((1,1000))
    goal=tf.convert_to_tensor(goal)

    for i in range(steps):

        d = grad(cm(x,M)+x_t , goal , model)                
        x -= eta * (d + lam * (cm(x,M)))
        

    return cm(x,M)+x_t,cm(x,M)


In [ ]:
def sneaky_adversarial_sett( n, x_target, steps, eta, lam , model):

    x=np.random.sample((1,299,299,3))
    x=tf.convert_to_tensor(x,dtype='float32')
    x_t = (x_target + np.ones((x_target.shape)))/2
    x_t = tf.image.resize(x_t, (299, 299))
    #x = x[None, ...]
    goal = np.zeros((1000, 1))
    goal[n]=1    
    goal=goal.reshape((1,1000))
    goal=tf.convert_to_tensor(goal)

    for i in range(steps):

        d = grad(x+x_t , goal , model)                
        x -= eta * (d*0.1 + lam * x)
        

    return x+x_t,x


In [ ]:
adv_ex ,pret = sneaky_adversarial_sett(6, images[12] ,30 , 1 , 0.12  ,model_inception_v3 )
pred = model_inception_v3.predict(adv_ex)
print(np.argmax(pred))
print(pred.max())
#x = (adv_ex[0] + np.ones((adv_ex[0].shape)))/2
plt.imshow(adv_ex[0]) 
plt.show()
plt.imshow(pret[0]) 
plt.show()

In [ ]:
M = np.ones((1,224,224,3))

In [ ]:
def cm(X,M2):
    x = X.numpy()
    m = M2.numpy()
    for i in range(224):
        for j in range(224):
            for k in range(3):
                x[0,i,j,k] = m[0,i,j,k]*x[0,i,j,k]
    X = tf.convert_to_tensor(x,dtype='float32')
    return X

In [ ]:
def med(X):
    x = X.numpy()
    s = x.sum()
    for i in range(224):
        for j in range(224):
            for k in range(3):
                x[0,i,j,k] = x[0,i,j,k]
    X = tf.convert_to_tensor(x,dtype='float32')
    return X

In [ ]:
def TV(X):
    x = X.numpy()
    t = 0
    for i in range(224):
        for j in range(224):
            for k in range(3):
                t = t + ((x[0,i,j,k]-x[0,i,j-1,k])**2+(x[0,i-1,j,k]-x[0,i,j,k])**2)**(1/2)
    t = tf.convert_to_tensor(t,dtype='float')
    return t
                

In [ ]:
x = tf.image.resize(((images[176] + np.ones((images[176].shape)))/2), (224, 224))
x = x[None, ...]
print(np.argmax(model_MobilNetV2.predict(x)))
print(model_MobilNetV2.predict(x).max())

In [ ]:
i = 71
adv_ex ,pret = sneaky_adversarial(111, images[i] ,20 , 1 , 0.12  ,model_MobilNetV2 )
pred = model_MobilNetV2.predict(adv_ex)
print(np.argmax(pred))
print(pred.max())
#x = (adv_ex[0] + np.ones((adv_ex[0].shape)))/2
plt.imshow(adv_ex[0]) 
plt.show()
plt.imshow(pret[0]) 
plt.show()
im=cv2.blur(pret[0].numpy(), ksize=(2, 2))
plt.imshow(adv_ex[0]-pret[0]+im)
print(np.argmax(model_MobilNetV2.predict(adv_ex[0]-pret[0]+im.reshape((1,224,224,3)))))
print(model_MobilNetV2.predict(adv_ex[0]-pret[0]+im.reshape((1,224,224,3))).max())
plt.show()
print(true_classes[i] - 1)

In [ ]:
x = tf.image.resize(images[5], (224, 224))
x = x[None, ...]
print(np.argmax(model_ResNet101.predict(x)))
show_image(images[5])

In [ ]:
def augment_data():
    """
    n : integer
        number of adversarial examples to generate
    data : list of tuples
        data set to generate adversarial examples using
    """
    # Our augmented training set:
    augmented = []
       
    for i in range(200):
        # Progress "bar"
        if i % 2 == 0:
            print("Generated images: " + str(i))
            
        y_fake = np.random.randint(1000)
        
        adv_ex = sneaky_adversarial_FGSM(y_fake , images[i] , 0.05   , model_inception_v3 )
        
        y_actual = true_classes[i] - 1
        
        augmented.append(( adv_ex , y_fake , y_actual ))
        
    return augmented

In [ ]:
fakedata_FGSM_for_model_inception_v3=augment_data()

In [ ]:
def augment_data_L_BFGS():
    """
    n : integer
        number of adversarial examples to generate
    data : list of tuples
        data set to generate adversarial examples using
    """
    # Our augmented training set:
    augmented = []
       
    for i in range(200):
        # Progress "bar"
        if i % 10 == 0:
            print("Generated images: " + str(i))
            
        y_fake = np.random.randint(1000)
        
        adv_ex ,pret = sneaky_adversarial_sett(y_fake, images[i] ,30 , 1 , 0.12  ,model_inception_v3 )
        
        y_actual = true_classes[i] - 1
        
        augmented.append(( adv_ex , y_fake , y_actual ))
        
    return augmented

In [ ]:
def augment_data_I_FGSM():
    """
    n : integer
        number of adversarial examples to generate
    data : list of tuples
        data set to generate adversarial examples using
    """
    # Our augmented training set:
    augmented = []
       
    for i in range(200):
        # Progress "bar"
        if i % 10 == 0:
            print("Generated images: " + str(i))
            
        y_fake = np.random.randint(1000)
        
        adv_ex = sneaky_adversarial_MI_FGSM(y_fake, images[i] , 10 , 0.05 , 0  ,model_inception_v3 )
        
        y_actual = true_classes[i] - 1
        
        augmented.append(( adv_ex , y_fake , y_actual ))
        
    return augmented

In [ ]:
def augment_data_MI_FGSM():
    """
    n : integer
        number of adversarial examples to generate
    data : list of tuples
        data set to generate adversarial examples using
    """
    # Our augmented training set:
    augmented = []
       
    for i in range(200):
        # Progress "bar"
        if i % 10 == 0:
            print("Generated images: " + str(i))
            
        y_fake = np.random.randint(1000)
        
        adv_ex = sneaky_adversarial_MI_FGSM(y_fake, images[i] , 10 , 0.05 , 1  ,model_inception_v3 )
        
        y_actual = true_classes[i] - 1
        
        augmented.append(( adv_ex , y_fake , y_actual ))
        
    return augmented

In [ ]:
fakedata_MI_FGSM = augment_data_MI_FGSM()

In [ ]:
fakedata_L_BFGS = augment_data_L_BFGS()

In [ ]:
fakedata_I_FGSM = augment_data_I_FGSM()

In [ ]:
def accuracy( test_data):
    tot = 200
    correct1 = 0
    correct2 = 0
    correct3 = 0
    correct4 = 0
    correct5 = 0
    for i in range(200):
        if i % 20 == 0:
            print(i)
        label1 = np.argmax(model_xception.predict(tf.image.resize(test_data[i][0], (299, 299))))
        label2 = np.argmax(model_InceptionResNetV2.predict(tf.image.resize(test_data[i][0], (299, 299))))
        label3 = np.argmax(model_inception_v3.predict(tf.image.resize(test_data[i][0], (299, 299))))
        label4 = np.argmax(model_MobilNetV2.predict(tf.image.resize(test_data[i][0], (224, 224))))
        label5 = np.argmax(model_MobilNet.predict(tf.image.resize(test_data[i][0], (224, 224))))
        correct1 += int((label1) == test_data[i][2])
        correct2 += int((label2) == test_data[i][2])
        correct3 += int((label3) == test_data[i][2])
        correct4 += int((label4) == test_data[i][2])
        correct5 += int((label5) == test_data[i][2])
    print('Accuracy: model_xception '+str(correct1/tot))
    print('Accuracy: model_InceptionResNetV2 '+str(correct2/tot))
    print('Accuracy: model_inception_v3 '+str(correct3/tot))
    print('Accuracy: model_MobilNetV2 '+str(correct4/tot))
    print('Accuracy: model_MobilNet '+str(correct5/tot))
    return correct1 / tot , correct2 / tot , correct3 / tot , correct4 / tot, correct5 / tot

print('Accuracy: ' + str(accuracy(fakedata_MI_FGSM)))


In [ ]:
plt.imshow(fakedata_L_BFGS[2][0].numpy().reshape((299,299,3)))

In [ ]:
plt.imshow(fakedata_MI_FGSM[101][0][0])
print(np.max(model_MobilNetV2.predict(fakedata_MI_FGSM[101][0])))
toster = tf.image.resize(images[101], (224, 224))
toster = toster[None, ...]
print(np.max(model_MobilNetV2.predict(toster)))
print(np.argmax(model_MobilNetV2.predict(toster)))
print(fakedata_MI_FGSM[101][1])
print(fakedata_MI_FGSM[101][2])

In [ ]:
cd nips-2017-adversarial-learning-development-set

In [ ]:
import os
from io import BytesIO
import IPython.display
import numpy as np
import pandas as pd
from PIL import Image
import scipy
import imageio
tensorflow_master = ""
input_dir         = "../nips-2017-adversarial-learning-development-set/images/"
max_epsilon       = 16.0
image_width       = 299
image_height      = 299
batch_size        = 200

eps = 2.0 * max_epsilon / 255.0
batch_shape = [batch_size, image_height, image_width, 3]
num_classes = 1001

def load_images(input_dir, batch_shape):
    images = np.zeros(batch_shape)
    filenames = []
    idx = 0
    batch_size = batch_shape[0]
    for filepath in sorted(tfv1.gfile.Glob(os.path.join(input_dir, '*.png'))):
        with tfv1.gfile.Open(filepath, "rb") as f:
            images[idx, :, :, :] = imageio.imread(f, pilmode='RGB').astype(np.float)*2.0/255.0 - 1.0
        filenames.append(os.path.basename(filepath))
        idx += 1
        if idx == batch_size:
            yield filenames, images
            filenames = []
            images = np.zeros(batch_shape)
            idx = 0
    if idx > 0:
        yield filenames, images

def show_image(a, fmt='png'):
    a = np.uint8((a+1.0)/2.0*255.0)
    f = BytesIO()
    Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

class InceptionModel(object):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.built = False

    def __call__(self, x_input):
        """Constructs model and return probabilities for given input."""
        reuse = True if self.built else None
        with slim.arg_scope(inception.inception_v3_arg_scope()):
            _, end_points = inception.inception_v3(
                            x_input, num_classes=self.num_classes, is_training=False,
                            reuse=reuse)
        self.built = True
        output = end_points['Predictions']
        probs = output.op.inputs[0]
        return probs

In [ ]:
import tensorflow.compat.v1 as tfv1  # for TF 2

#tf.disable_v2_behavior()  # for TF 2
categories = pd.read_csv("categories.csv")
image_classes = pd.read_csv("images.csv")
image_iterator = load_images(input_dir, batch_shape)

# get first batch of images
filenames, images = next(image_iterator)

image_metadata = pd.DataFrame({"ImageId": [f[:-4] for f in filenames]}).merge(image_classes,
                                                                              on="ImageId")
true_classes = image_metadata["TrueLabel"].tolist()
target_classes = true_labels = image_metadata["TargetClass"].tolist()
true_classes_names = (pd.DataFrame({"CategoryId": true_classes})
                        .merge(categories, on="CategoryId")["CategoryName"].tolist())
target_classes_names = (pd.DataFrame({"CategoryId": target_classes})
                          .merge(categories, on="CategoryId")["CategoryName"].tolist())

print("Here's an example of one of the images in the development set")
show_image(images[7])

In [ ]:

true_classes_names[0]

In [ ]:
show_image(images[101])